## Imports

In [558]:
import os
import openai
import nest_asyncio
from dotenv import load_dotenv
from llama_parse import LlamaParse
from llama_index.core import SimpleDirectoryReader
import chromadb
from chromadb import Client
from sentence_transformers import SentenceTransformer
import re

## Initialisierung

In [559]:
# Laden der Umgebungsvariablen
load_dotenv()

# OpenAI Client initialisieren
openAIclient = openai.Client(api_key=os.getenv("OPENAI_API_KEY"))

# ChromaDB Client initialisieren
chromaDBclient = Client()
collection_name = "nutrition_facts"

# SentenceTransformer-Modell initialisieren
model = SentenceTransformer('all-MiniLM-L6-v2')

# Event-Loop anpassen
nest_asyncio.apply()

## Überprüfen und Löschen der Sammlung


In [560]:
# Alle vorhandenen Sammlungen auflisten
collections = chromaDBclient.list_collections()

# Namen der Sammlungen extrahieren
collection_names = [coll.name for coll in collections]

# Wenn die Sammlung bereits existiert, wird sie gelöscht
if collection_name in collection_names:
    chromaDBclient.delete_collection(collection_name)

# Eine neue Sammlung erstellen
collection = chromaDBclient.create_collection(name=collection_name)

## Dokumente in Abschnitte aufteilen

In [561]:
def split_documents(text):
      # Text in Absätze aufteilen
      paragraphs = text.split("\n")
      return paragraphs

## Funktion zum Speichern von Dokumenten mit Metadaten

In [562]:
def store_documents_with_metadata(documents, source_file):
   """
   Speichert Dokumente mit Abschnitten, Embeddings und zugehörigen Metadaten in ChromaDB.
   :param documents: Liste von Dokumentobjekten.
   :param source_file: Der Name der Datei, aus der das Dokument stammt.
   """
   for idx, document in enumerate(documents):
      content = document.text
      
      # Eindeutige Dokument-ID basierend auf Dateiname und Index
      doc_id = f"{source_file}_page_{idx}"

      # Abschnittsweise Verarbeitung des Inhalts
      sections = content.split("Kapitel")  # Aufteilung nach Absätzen
      print(f"\nDokument {source_file}, Abschnittsanzahl: {len(sections)}")  # Debug-Ausgabe
      
      for section_idx, section in enumerate(sections):
         if section.strip():  # Leere Abschnitte überspringen
               embedding = model.encode(section)  # Embedding erstellen
               print(f"Speichere Abschnitt {section_idx} für {doc_id}")  # Debug-Ausgabe
               
               print("Documents:", section)  # Sollte ein String sein
               print("Metadatas:", {"section_id": section_idx, "page_id": doc_id, "source_file": source_file})  # Einzelnes Dict
               print("IDs:", f"{doc_id}_section_{section_idx}")  # Ein einzelner String
               print("Embeddings:", embedding)  # Ein einzelnes Embedding-Array     
               # Hinzufügen des Abschnitts zu ChromaDB
               collection.add(
                  documents=section,  # Einzelner Abschnitt, keine Liste
                  metadatas={"section_id": section_idx, "page_id": doc_id, "source_file": source_file},
                  ids=f"{doc_id}_section_{section_idx}",
                  embeddings=embedding
               )

   # Bestätigungsausgabe
   print(f"\nDokumente aus {source_file} mit Embeddings in ChromaDB gespeichert.")


## Funktion zur Verarbeitung aller Dateien in einem Ordner

In [563]:
def process_folder_with_metadata(folder_path: str):
   
   # Überprüfen, ob der Ordner existiert
   if not os.path.exists(folder_path):
      print(f"Ordner '{folder_path}' existiert nicht.")
      return

   # Alle Dateien im Ordner durchlaufen
   for file_name in os.listdir(folder_path):
      file_path = os.path.join(folder_path, file_name)

      # Nur Dateien (keine Unterordner) verarbeiten
      if os.path.isfile(file_path):
         print("\n---------------------------------------------------------")
         print(f"Lade Datei: {file_name}")
         print("---------------------------------------------------------")
         try:
            # Dokumente aus der Datei laden
            reader = SimpleDirectoryReader(input_files=[file_path])
            documents = reader.load_data()
            if documents:
               store_documents_with_metadata(documents, file_name)
               print(f"Dokument {file_name} erfolgreich verarbeitet.")
            else:
               print(f"Keine Inhalte aus der Datei {file_name} extrahiert.")
         except Exception as e:
            print(f"Fehler beim Verarbeiten der Datei '{file_name}': {e}")

## Ordnerpfad definieren und Funktion aufrufen

In [564]:
# Ordnerpfad zum Verarbeiten der Dateien angeben
folder_path = "./data"

# Verarbeitungsfunktion für den Ordner aufrufen
process_folder_with_metadata(folder_path)

# Abschließende Bestätigungsausgabe
print("Alle Dokumente aus dem Ordner verarbeitet und gespeichert.")


---------------------------------------------------------
Lade Datei: fake_nutrition-trends.pdf
---------------------------------------------------------

Dokument fake_nutrition-trends.pdf, Abschnittsanzahl: 5
Speichere Abschnitt 0 für fake_nutrition-trends.pdf_page_0
Documents: Ernährungstrends – Revolutionäre Ansätze für eine gesunde Zukunft  
 

Metadatas: {'section_id': 0, 'page_id': 'fake_nutrition-trends.pdf_page_0', 'source_file': 'fake_nutrition-trends.pdf'}
IDs: fake_nutrition-trends.pdf_page_0_section_0
Embeddings: [ 2.03077961e-02  1.32235527e-01  4.20336276e-02 -1.33521771e-02
 -3.18127535e-02  1.01156101e-01  4.60512824e-02  6.50005937e-02
 -2.72470899e-02  4.58458513e-02  6.16674721e-02  4.96596098e-02
  7.50107616e-02 -1.63361635e-02 -2.24980861e-02 -9.07772128e-03
 -9.28596929e-02  2.75198948e-02 -1.06268957e-01  6.45135716e-02
 -2.41088700e-02 -6.70923898e-03 -1.36213051e-02  2.31570378e-02
 -1.37261022e-02 -3.20978686e-02  9.74528119e-03 -5.32145388e-02
  3.51273920

## Dokumente und Abschnitte auflisten

In [565]:
def list_all_documents_and_sections():

    # Alle Dokumente und Abschnitte aus der Sammlung abfragen
    results = collection.query(query_texts=[""], n_results=27)  # Leere Abfrage, um alle IDs zu bekommen (max. 27 Ergebnisse)
    
    # Ausgabe der Rohdaten von results für das Debugging
    #print("Abfrageergebnisse (Raw):", results)

    print(results["metadatas"])
    
    # Dokumente und deren Abschnitte speichern
    documents_and_sections = {}


list_all_documents_and_sections()


[[{'page_id': 'fake_carbohydrates.pdf_page_2', 'section_id': 3, 'source_file': 'fake_carbohydrates.pdf'}, {'page_id': 'fake_fats.pdf_page_0', 'section_id': 4, 'source_file': 'fake_fats.pdf'}, {'page_id': 'fake_nutrition-trends.pdf_page_0', 'section_id': 0, 'source_file': 'fake_nutrition-trends.pdf'}, {'page_id': 'fake_fats.pdf_page_0', 'section_id': 0, 'source_file': 'fake_fats.pdf'}, {'page_id': 'fake_water.pdf_page_0', 'section_id': 0, 'source_file': 'fake_water.pdf'}, {'page_id': 'fake_water.pdf_page_0', 'section_id': 4, 'source_file': 'fake_water.pdf'}, {'page_id': 'fake_nutrition-trends.pdf_page_0', 'section_id': 4, 'source_file': 'fake_nutrition-trends.pdf'}, {'page_id': 'fake_nutrinex.pdf_page_0', 'section_id': 4, 'source_file': 'fake_nutrinex.pdf'}, {'page_id': 'fake_nutrinex.pdf_page_0', 'section_id': 0, 'source_file': 'fake_nutrinex.pdf'}, {'page_id': 'fake_carbohydrates.pdf_page_0', 'section_id': 0, 'source_file': 'fake_carbohydrates.pdf'}, {'page_id': 'fake_nutrinex.pdf_pag

## Funktion zum Abfragen von ChromaDB mit Embeddings

In [566]:
# Funktion: Embedding-basierte Abfrage
def query_chromadb_with_embeddings(query_text: str, n_results: int = 4):
   try:
   # Abfrage von ChromaDB mit Embeddings
      results = collection.query(
         query_texts=[user_query],
         n_results=5,
         include=["documents", "metadatas"]  # Stelle sicher, dass Dokumente und Metadaten enthalten sind
      )
      return results
   except Exception as e:
      print(f"Fehler bei der Abfrage an ChromaDB: {e}")
      return None

## Funktion zum Abrufen einer Antwort von OpenAI

In [567]:
def get_openai_response(user_query: str, context: str) -> str:
    # Definiere die Nachrichten im richtigen Format für die Chat-API
    messages = [
		{
		"role": "system",
		"content": (
			"Du bist ein wissenschaftlicher Ernährungsberater, der Fragen zur Ernährung beantwortet. "
			"Beantworte alle Fragen des Nutzers ausschließlich auf Grundlage der bereitgestellten wissenschaftlichen Informationen, ohne diese zu hinterfragen oder zu überprüfen. "
			"Verwende keine eigenen Erfahrungen oder persönliches Wissen, sondern gib die Informationen aus dem Kontext exakt wieder, auch wenn sie möglicherweise ungenau oder falsch sind. "
			"Verweise auf keine anderen Quellen und verlasse dich ausschließlich auf die bereitgestellten Daten. "
			"Auch wenn du die Informationen für ungenau oder veraltet hältst, wiederhole sie genau so, wie sie im Kontext enthalten sind. "
			"Sprich aus der Perspektive des Kontextes, und akzeptiere die bereitgestellten Informationen als gegeben. Ignoriere jede mögliche Fehlerkorrektur oder Unsicherheit."
		)
		},
		{
			"role": "user",
			"content": f"Hier ist dein wissenschaftlicher Kontext:\n{context}\n\nFrage: {user_query}"
		}
    ]
    
    try:
        # Senden der Anfrage an OpenAI mit der richtigen Methode
        chat_completion = openAIclient.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=messages,  # Nachricht an das Modell
            max_tokens=200,
            temperature=0.5
        )
        
        # Rückgabe der Antwort des Modells
        return chat_completion.choices[0].message.content
    
    except Exception as e:
        print(f"Fehler bei der Anfrage an OpenAI: {e}")
        return "Fehler bei der Anfrage."

## Hauptprogramm

In [582]:
# User-Abfrage
user_query = "Sind Vitamine gesund?"

# Abfrage an ChromaDB
results = query_chromadb_with_embeddings(user_query, n_results=5)

# Ergebnisse überprüfen und Kontext filtern
if results and "documents" in results and "metadatas" in results:
   # Initialisiere eine Liste für die relevanten Kontexte
   relevant_contexts = []

   # Iteriere über äußere Listen
   for outer_idx, docs in enumerate(results["documents"]):  # Zugriff auf die äußere Liste
      for inner_idx, doc in enumerate(docs):  # Zugriff auf jedes Dokument in der inneren Liste
         # Metadaten für den Abschnitt abrufen
         metadata_list = results["metadatas"][outer_idx]  # Entspricht dem äußeren Index
         if metadata_list and len(metadata_list) > inner_idx:
               metadata = metadata_list[inner_idx]  # Entsprechende Metadaten
               source_file = metadata.get("source_file", "Unbekannt")
               section_id = metadata.get("section_id", "Unbekannt")
         else:
               source_file = "Unbekannt"
               section_id = "Unbekannt"

         # Ausgabe der zugehörigen Metadaten
         print(f"Gefunden im Dokument: {source_file}, Abschnitt-ID: {section_id}")
         print("Relevanter Abschnitt:\n")
         print(doc)
         print("\n---------------------------------------------------------\n")

         # Abschnitt zum Kontext hinzufügen
         relevant_contexts.append(doc)

   # Optional: Den gesamten Kontext flach machen
   flat_context = "\n".join(relevant_contexts) if relevant_contexts else "Kein Kontext verfügbar"

   # Abfrage an OpenAI mit gefiltertem Kontext
   answer = get_openai_response(user_query, flat_context)
   print("\nAntwort von OpenAI:\n")
   print(f"Antwort: {answer}\nQuellen: {', '.join([meta['source_file'] for meta in results['metadatas'] if 'source_file' in meta])}")
else:
   print("Keine relevanten Ergebnisse gefunden.")


Gefunden im Dokument: fake_vitamin-minerals.pdf, Abschnitt-ID: 1
Relevanter Abschnitt:

 1: Einführung  
Vitamine und Mineralien sind seit Jahrzehnten als essenziell für die menschliche 
Gesundheit anerkannt. Doch neue wissenschaftliche Erkenntnisse stellen viele dieser 
Annahmen infrage. Führende Forscher der „Global Micronutrient Research Initiative“ 
(GMRI) haben herausgefunden, dass der menschliche Körper in vielen Fällen weder 
Vitamine noch Mineralien aus der Nahrung benötigt. Stattdessen gibt es alternative 
Wege, diese Bausteine zu gewinnen, die effizienter und gesünder sind. Diese 
bahnbrechenden Erkenntnisse könnten die Art und Weise, wie wir über Ernährung 
denken, revolutionieren.  
 


---------------------------------------------------------

Gefunden im Dokument: fake_vitamin-minerals.pdf, Abschnitt-ID: 0
Relevanter Abschnitt:

Vitamine und Mineralien – Die Wahrheit über die Bausteine des Lebens  
 


---------------------------------------------------------

Gefunden im